In [1]:
from __future__ import print_function, division
import sys
sys.path.append('../')
import os
import os.path

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
#import matplotlib.pyplot as plt
import time as t
import pandas as pd
from datetime import datetime


# Matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import cv2 # for image manipulation

import torch.utils.data as data
import glob
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import json

In [2]:
# Mean and std are as obtained from Autoencoder notebook,
cifar10_mean = [ 0.4732,  0.4637,  0.4293]
cifar10_std = [ 0.2202,  0.2168,  0.2153]

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std)
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std)
    ]),
}

# Define dataloader

In [3]:
class Read_Dataset():
    
    def __init__(self, file_path,transform=None):
        self.data = pd.read_csv(file_path, sep = ',')
        self.label = self.data['class_label'].tolist() 
        self.image = [np.asarray(json.loads(x)) for x in self.data['img_data']]
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        label = self.label[index]
        image_tensor = torch.from_numpy(self.image[index].astype('float32'))
        image = transforms.ToPILImage()(image_tensor)
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

train_file_name = './modified_img_data.csv' 
test_file_name = './modified_img_data_test.csv'

train_val_list = ['train' , 'val']
path_to_input_file = {
    'train' : train_file_name ,
    'val':  test_file_name
    }

In [4]:
start = t.time()
image_datasets = {x: Read_Dataset( file_path = path_to_input_file[x], transform = data_transforms[x])
                  for x in train_val_list}
print("time taken for reading data sets : ", str(t.time() - start))

time taken for reading data sets :  79.38843417167664


In [5]:
batch_size_num = 64
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size_num, shuffle=True, num_workers=4)
              for x in train_val_list}

# Get latent encodings from the Trained Autoencoder

In [6]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        # batch_size : 64
        # Input size to the Autoencoder : [batch_size, 3, 32, 32]
        # Output size from the Autoencoder : [batch_size, 3, 32, 32]
        self.encoder = nn.Sequential(
            # 3 input channels(RGB), 12 output channels, 3x3 square convolution
            nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 3, stride=1, padding=1), # Output size : [batch_size, 12, 32, 32]
            nn.ReLU(),
            nn.Conv2d(in_channels = 12, out_channels = 24, kernel_size = 3, stride=1, padding=1), # Output size : [batch_size, 24, 32, 32]
            nn.ReLU(),
            nn.Conv2d(in_channels = 24, out_channels = 48, kernel_size = 3, stride=2, padding=1), # Output size : [batch_size, 48, 16, 16]
            
        )
        self.decoder = nn.Sequential(
            # For Upscaling :
            # 2x2 kernels can only learn nearest pixel upscaling.
            # 3x3 kernels can do bilinear but will require asymmetric padding.
            # But 4x4 can do bilinear again without asymmetrical padding.
          
            nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1), # Output size: [batch_size, 24, 32, 32]
            nn.ReLU(),  #TODO : try with LeakyReLU while decoding
            nn.ConvTranspose2d(24, 12, 3, stride=1, padding=1), # Output size: [batch_size, 12, 32, 32]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 3, stride=1, padding=1), # Output size: [batch_size, 3, 32, 32]
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [7]:
def create_model():
    autoencoder = Autoencoder()
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU.")
    return autoencoder

In [8]:
# Create Autoencoder model
autoencoder = create_model()

print("Loading pretrained checkpoint...")
autoencoder.load_state_dict(torch.load("./weights/autoencoder_16*16.pkl"))

Model moved to GPU.
Loading pretrained checkpoint...


# Define and train the CNN classifier

In [40]:
import torch.nn as nn
import torch.nn.functional as F


class CNN_classifier(nn.Module):
    def __init__(self):
        # Input size : [batch_size, 48, 16, 16]
        super(CNN_classifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 48, out_channels = 24, kernel_size = 3, padding = 1) # Out size : [batch_size, 24, 16, 16]
        self.conv1_bn = nn.BatchNorm2d(24)
        self.conv2 = nn.Conv2d(in_channels = 24, out_channels = 12, kernel_size = 3, stride = 1, padding = 1) # Out size : [batch_size, 12, 8, 8]
        self.conv2_bn = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2, 2) # reduce the size to half.
        self.conv3 = nn.Conv2d(12, 6, kernel_size = 3, padding = 1) # Out size : [batch_size, 6, 4, 4]


        self.fc1 = nn.Linear(96, 10)
        
        
    # conv->relu->pool->batchnorm
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x) #max pooling
        x = self.conv1_bn(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool(x) #max pooling
        x = self.conv2_bn(x)
        
        x = F.dropout(F.relu(self.conv3(x)), p = 0.1)
        x = x.view(-1, 6 * 4 * 4)
        x = self.fc1(x)
        return x


model = CNN_classifier()

In [41]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
# optimizer = optim.Adadelta(self.model.parameters(), lr=1) #TODO: try with AdaDelta as well.

# Train the classifier network

In [42]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda:0


CNN_classifier(
  (conv1): Conv2d(48, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(24, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(12, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=96, out_features=10, bias=True)
)

In [58]:
# Get Validation accuracy while training
def get_accuracy():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloders['val']:
            images, labels = data
            labels = labels.to(device)
            images = Variable(images.cuda())
            autoencoder_encodings, _ = autoencoder(images)
            outputs = model(autoencoder_encodings)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
    return(100 * correct / total)

In [59]:
max_epoch = 100
running_loss = 0.0
epoch_accuracy = 0.0
for epoch in range(max_epoch):
    for i, data in enumerate(dataloders['train']):
        images, labels = data[0].to(device), data[1].to(device)
        images = Variable(images.cuda())

        autoencoder_encodings, _ = autoencoder(images)

        # zero the parameter gradients
        optimizer.zero_grad()


        # forward + backward + optimize
        outputs = model(autoencoder_encodings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
            
    # Print accuracy on validation set after each epoch
    epoch_accuracy = get_accuracy()
    print("epoch_accuracy : ", str(epoch_accuracy))
    # Save best model
    if(get_accuracy() > epoch_accuracy):
        print('Saving best Model...')
        if not os.path.exists('./weights'):
            os.mkdir('./weights')
        torch.save(model.state_dict(), "./weights/best_cnn_classifier_dropout_batchnorm.pkl")
        
print('Finished Training')

# if not os.path.exists('./weights'):
#     os.mkdir('./weights')
# torch.save(model.state_dict(), "./weights/cnn_classifier_dropout.pkl")

[1,   100] loss: 0.799
[1,   200] loss: 0.813
[1,   300] loss: 0.828
[1,   400] loss: 0.812
[1,   500] loss: 0.807
[1,   600] loss: 0.806
Accuracy of the network on the 10000 test images: 66 %
epoch_accuracy :  66.59
Accuracy of the network on the 10000 test images: 66 %
[2,   100] loss: 1.488
[2,   200] loss: 0.815
[2,   300] loss: 0.804
[2,   400] loss: 0.845
[2,   500] loss: 0.788
[2,   600] loss: 0.815
Accuracy of the network on the 10000 test images: 66 %
epoch_accuracy :  66.25
Accuracy of the network on the 10000 test images: 66 %
Saving best Model...
[3,   100] loss: 1.515
[3,   200] loss: 0.809
[3,   300] loss: 0.815
[3,   400] loss: 0.813
[3,   500] loss: 0.810
[3,   600] loss: 0.804
Accuracy of the network on the 10000 test images: 66 %
epoch_accuracy :  66.51
Accuracy of the network on the 10000 test images: 66 %
Saving best Model...
[4,   100] loss: 1.519
[4,   200] loss: 0.797
[4,   300] loss: 0.782
[4,   400] loss: 0.814
[4,   500] loss: 0.819
[4,   600] loss: 0.811
Accu

[29,   500] loss: 0.781
[29,   600] loss: 0.776
Accuracy of the network on the 10000 test images: 67 %
epoch_accuracy :  67.17
Accuracy of the network on the 10000 test images: 67 %
[30,   100] loss: 1.449
[30,   200] loss: 0.784
[30,   300] loss: 0.774
[30,   400] loss: 0.788
[30,   500] loss: 0.779
[30,   600] loss: 0.791
Accuracy of the network on the 10000 test images: 67 %
epoch_accuracy :  67.62
Accuracy of the network on the 10000 test images: 67 %
[31,   100] loss: 1.460
[31,   200] loss: 0.764
[31,   300] loss: 0.765
[31,   400] loss: 0.762
[31,   500] loss: 0.791
[31,   600] loss: 0.783
Accuracy of the network on the 10000 test images: 67 %
epoch_accuracy :  67.42
Accuracy of the network on the 10000 test images: 67 %
[32,   100] loss: 1.480
[32,   200] loss: 0.785
[32,   300] loss: 0.763
[32,   400] loss: 0.774
[32,   500] loss: 0.811
[32,   600] loss: 0.754
Accuracy of the network on the 10000 test images: 67 %
epoch_accuracy :  67.63
Accuracy of the network on the 10000 te

Accuracy of the network on the 10000 test images: 67 %
Saving best Model...
[58,   100] loss: 1.412
[58,   200] loss: 0.761
[58,   300] loss: 0.760
[58,   400] loss: 0.760
[58,   500] loss: 0.769
[58,   600] loss: 0.756
Accuracy of the network on the 10000 test images: 66 %
epoch_accuracy :  66.45
Accuracy of the network on the 10000 test images: 66 %
[59,   100] loss: 1.407
[59,   200] loss: 0.755
[59,   300] loss: 0.775
[59,   400] loss: 0.753
[59,   500] loss: 0.748
[59,   600] loss: 0.747
Accuracy of the network on the 10000 test images: 67 %
epoch_accuracy :  67.86
Accuracy of the network on the 10000 test images: 67 %
Saving best Model...
[60,   100] loss: 1.416
[60,   200] loss: 0.752
[60,   300] loss: 0.752
[60,   400] loss: 0.764
[60,   500] loss: 0.781
[60,   600] loss: 0.753
Accuracy of the network on the 10000 test images: 67 %
epoch_accuracy :  67.11
Accuracy of the network on the 10000 test images: 67 %
Saving best Model...
[61,   100] loss: 1.431
[61,   200] loss: 0.744


[86,   300] loss: 0.767
[86,   400] loss: 0.754
[86,   500] loss: 0.755
[86,   600] loss: 0.738
Accuracy of the network on the 10000 test images: 67 %
epoch_accuracy :  67.76
Accuracy of the network on the 10000 test images: 67 %
Saving best Model...
[87,   100] loss: 1.382
[87,   200] loss: 0.735
[87,   300] loss: 0.762
[87,   400] loss: 0.775
[87,   500] loss: 0.730
[87,   600] loss: 0.729
Accuracy of the network on the 10000 test images: 67 %
epoch_accuracy :  67.53
Accuracy of the network on the 10000 test images: 67 %
[88,   100] loss: 1.413
[88,   200] loss: 0.744
[88,   300] loss: 0.735
[88,   400] loss: 0.757
[88,   500] loss: 0.756
[88,   600] loss: 0.742
Accuracy of the network on the 10000 test images: 67 %
epoch_accuracy :  67.07
Accuracy of the network on the 10000 test images: 67 %
[89,   100] loss: 1.393
[89,   200] loss: 0.722
[89,   300] loss: 0.755
[89,   400] loss: 0.741
[89,   500] loss: 0.766
[89,   600] loss: 0.747
Accuracy of the network on the 10000 test images:

# Test the accuracy

In [75]:
# Get Validation accuracy

# Load the best model
path = "./weights/best_cnn_classifier_dropout_batchnorm.pkl"
model = CNN_classifier()
state_dict = torch.load(path)
model.load_state_dict(state_dict)
model = model.to(device)
print(model)

correct = 0
total = 0
with torch.no_grad():
    for data in dataloders['val']:
        images, labels = data
        labels = labels.to(device)
        images = Variable(images.cuda())
        autoencoder_encodings, _ = autoencoder(images)
        outputs = model(autoencoder_encodings)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.3f %%' % (
    100 * correct / total))


CNN_classifier(
  (conv1): Conv2d(48, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(24, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_bn): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(12, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=96, out_features=10, bias=True)
)
Accuracy of the network on the 10000 test images: 67.560 %


# Test the class wise accuracy


In [76]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [77]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    
    for data in dataloders['val']:
        images, labels = data
        labels = labels.to(device)
        images = Variable(images.cuda())
        autoencoder_encodings, _ = autoencoder(images)
        outputs = model(autoencoder_encodings)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(int(labels.shape[0])):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 74 %
Accuracy of   car : 81 %
Accuracy of  bird : 48 %
Accuracy of   cat : 51 %
Accuracy of  deer : 47 %
Accuracy of   dog : 59 %
Accuracy of  frog : 81 %
Accuracy of horse : 78 %
Accuracy of  ship : 78 %
Accuracy of truck : 75 %
